# <center> Flower Recognition Through CNN Keras

<a id="content1"></a>
# 1. Importing Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from keras.utils import to_categorical
import tensorflow as tf
import cv2
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set Random Seeds
np.random.seed(42)
rn.seed(42)
tf.set_random_seed(42)

In [ ]:
## Define Functions to Extract Training and Validation Sets
flower_images = []
flower_labels = []
IMG_SIZE = 150
daisy_dir = '../input/flowers-recognition/flowers/daisy'
sunflower_dir = '../input/flowers-recognition/flowers/sunflower'
tulip_dir = '../input/flowers-recognition/flowers/tulip'
rose_dir = '../input/flowers-recognition/flowers/rose'

In [ ]:
def assign_label(img, flower_type):
    return flower_type

In [ ]:
def load_training_data(flower_type, DIR):
    for img in tqdm(os.listdir(DIR)):
        label = assign_label(img, flower_type)
        path = os.path.join(DIR, img)
        img_data = cv2.imread(path, cv2.IMREAD_COLOR)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        flower_images.append(np.array(img_data))
        flower_labels.append(str(label))

In [ ]:
load_training_data('Daisy', daisy_dir)
print(len(flower_images))
load_training_data('Sunflower', sunflower_dir)
print(len(flower_images))
load_training_data('Tulip', tulip_dir)
print(len(flower_images))
load_training_data('Rose', rose_dir)
print(len(flower_images))
print(len(flower_images))
print(len(flower_labels))
print(set(flower_labels))

In [ ]:
## Visualize Some Random Images
fig, ax = plt.subplots(5, 3)
fig.set_size_inches(15, 15)
for i in range(5):
    for j in range(3):
        idx = rn.randint(0, len(flower_labels))
        ax[i, j].imshow(flower_images[idx])
        ax[i, j].set_title('Flower: ' + flower_labels[idx])

plt.tight_layout()

In [ ]:
print("Number of Pixels in each image:", 150 * 150 * 3)

In [ ]:
## Label Encoding for Y array & One Hot Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(flower_labels)
one_hot_labels = to_categorical(encoded_labels, 4)
print(one_hot_labels.shape)
print(type(flower_images))
print(len(flower_images))
print(flower_images[1].shape)
flower_images_arr = np.array(flower_images)

In [ ]:
# Image Standardization [Scale 0-1]
flower_images_arr = flower_images_arr / 255

In [ ]:
## Build the CNN Model
flower_model = Sequential()
flower_model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(150, 150, 3)))
flower_model.add(MaxPooling2D(pool_size=(2, 2)))
flower_model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
flower_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
flower_model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
flower_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
flower_model.add(Flatten())
flower_model.add(Dense(512, activation='relu'))
flower_model.add(Dense(4, activation="softmax"))
flower_model.summary()

In [ ]:
## Compile the Keras Model
flower_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## Fit on the Training set and make predictions on the Validation set
batch_size = 256
epochs = 21
flower_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)
flower_model.evaluate(X_test, y_test)

In [ ]:
## Evaluate the Model Performance
plt.plot(range(len(flower_model.history.history['acc'])), flower_model.history.history['acc'])
plt.plot(range(len(flower_model.history.history['loss'])), flower_model.history.history['loss'])
plt.title('Model - Accuracy Vs Loss')
plt.xlabel('Epochs')
plt.legend(['Accuracy', 'Loss'])
plt.show()

plt.plot(flower_model.history.history['loss'])
plt.plot(flower_model.history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'])
plt.show()

plt.plot(flower_model.history.history['acc'])
plt.plot(flower_model.history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
## Visualize Predictions on the Validation Set
predictions = flower_model.predict(X_test)
print("Predicted Probabilities -\n", predictions[:4])
predicted_classes = np.argmax(predictions, axis=1)
print("\nPredicted Class [Highest Prob] -", predicted_classes[:4])

In [ ]:
# Store some properly as well as misclassified indexes
i = 0
correctly_classified = []
misclassified = []

for i in range(len(y_test)):
    if(np.argmax(y_test[i]) == predicted_classes[i]):
        correctly_classified.append(i)
    if(len(correctly_classified) == 8):
        break

i = 0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i]) == predicted_classes[i]):
        misclassified.append(i)
    if(len(misclassified) == 8):
        break
count = 0
fig, ax = plt.subplots(4, 2)
fig.set_size_inches(15, 15)
for i in range(4):
    for j in range(2):
        ax[i, j].imshow(X_test[correctly_classified[count]])
        ax[i, j].set_title("Predicted Flower: " + str(label_encoder.inverse_transform([predicted_classes[correctly_classified[count]]])) +
                           "\n" + "Actual Flower: " + str(label_encoder.inverse_transform(np.argmax([y_test[correctly_classified[count]]]))))
        plt.tight_layout()
        count += 1

In [ ]:

# Display Misclassified Images of Flowers
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count = 0
fig, ax = plt.subplots(4, 2)
fig.set_size_inches(15, 15)
for i in range(4):
    for j in range(2):
        ax[i, j].imshow(X_test[misclassified[count]])
        ax[i, j].set_title("Predicted Flower: " + str(label_encoder.inverse_transform([predicted_classes[misclassified[count]]])) +
                           "\n" + "Actual Flower: " + str(label_encoder.inverse_transform(np.argmax([y_test[misclassified[count]]]))))
        plt.tight_layout()
        count += 1